# Multi-Turn OpenCode Session Management

This notebook manages persistent multi-turn conversations with OpenCode CLI,
maintaining conversation context and history across multiple executions.

**Parameters:**
- `session_id`: Unique identifier for the conversation session
- `prompt`: New prompt to add to the conversation
- `model`: OpenCode model to use (optional)
- `action`: Session action (start, continue, summarize, end)
- `max_history`: Maximum conversation history to maintain

In [ ]:
# Default parameters - will be overridden by papermill
session_id = "default_session"
prompt = "Hello, let's start a conversation"
model = ""
action = "continue"  # start, continue, summarize, end
max_history = 10
verbose = "false"

In [ ]:
#!/bin/bash

# Setup session management environment
echo "=== Multi-Turn OpenCode Session Management ==="
echo "Session ID: $session_id"
echo "Action: $action"
echo "New prompt: $prompt"
echo "Model: ${model:-default}"
echo "Max history: $max_history"
echo "Verbose: $verbose"
echo

# Create session directory structure
SESSION_DIR="sessions/${session_id}"
mkdir -p "$SESSION_DIR"

# Session files
HISTORY_FILE="$SESSION_DIR/conversation_history.md"
METADATA_FILE="$SESSION_DIR/session_metadata.json"
CONTEXT_FILE="$SESSION_DIR/current_context.txt"

echo "Session directory: $SESSION_DIR"
echo "History file: $HISTORY_FILE"
echo "Metadata file: $METADATA_FILE"
echo

In [ ]:
#!/bin/bash

# Initialize or load existing session
echo "=== Session Initialization ==="

TIMESTAMP=$(date '+%Y-%m-%d %H:%M:%S')

if [ "$action" = "start" ] || [ ! -f "$METADATA_FILE" ]; then
    echo "Starting new session: $session_id"
    
    # Create session metadata
    cat > "$METADATA_FILE" << EOF
{
  "session_id": "$session_id",
  "created_at": "$TIMESTAMP",
  "last_updated": "$TIMESTAMP",
  "message_count": 0,
  "model": "${model:-default}",
  "max_history": $max_history,
  "status": "active"
}
EOF

    # Initialize conversation history
    cat > "$HISTORY_FILE" << EOF
# OpenCode Conversation Session: $session_id

**Created:** $TIMESTAMP  
**Model:** ${model:-default}  
**Max History:** $max_history  

---

EOF

    # Initialize context file
    echo "" > "$CONTEXT_FILE"
    
    echo "✅ New session initialized"
else
    echo "Loading existing session: $session_id"
    
    # Update last_updated timestamp
    if command -v jq >/dev/null 2>&1; then
        # Use jq if available for proper JSON handling
        jq --arg timestamp "$TIMESTAMP" '.last_updated = $timestamp' "$METADATA_FILE" > "${METADATA_FILE}.tmp" && mv "${METADATA_FILE}.tmp" "$METADATA_FILE"
    else
        # Fallback: simple sed replacement
        sed -i "s/\"last_updated\": \"[^\"]*\"/\"last_updated\": \"$TIMESTAMP\"/" "$METADATA_FILE"
    fi
    
    echo "✅ Session loaded"
fi

echo
echo "Current session metadata:"
cat "$METADATA_FILE"
echo

In [ ]:
#!/bin/bash

# Manage conversation history and context
echo "=== Managing Conversation Context ==="

# Count current messages in history
CURRENT_MESSAGES=$(grep -c "^## Message [0-9]" "$HISTORY_FILE" 2>/dev/null || echo "0")
NEW_MESSAGE_NUM=$((CURRENT_MESSAGES + 1))

echo "Current messages in history: $CURRENT_MESSAGES"
echo "New message number: $NEW_MESSAGE_NUM"

# Prepare context for OpenCode (last N messages)
CONTEXT_MESSAGES=$((max_history < CURRENT_MESSAGES ? max_history : CURRENT_MESSAGES))
echo "Including last $CONTEXT_MESSAGES messages as context"

if [ $CONTEXT_MESSAGES -gt 0 ]; then
    echo "Extracting conversation context..."
    
    # Extract the last N messages for context
    # This is a simplified extraction - in production you'd want more robust parsing
    tail -n +10 "$HISTORY_FILE" | head -n -5 > "$CONTEXT_FILE" 2>/dev/null || echo "" > "$CONTEXT_FILE"
    
    echo "Context prepared ($(wc -l < "$CONTEXT_FILE") lines)"
else
    echo "No previous context available"
    echo "" > "$CONTEXT_FILE"
fi

echo

In [ ]:
#!/bin/bash

# Execute OpenCode with conversation context
echo "=== Executing OpenCode with Session Context ==="

# Prepare the full prompt with context
FULL_PROMPT=""

# Add conversation context if available
if [ -s "$CONTEXT_FILE" ]; then
    echo "Including conversation context in prompt..."
    FULL_PROMPT="Previous conversation context:\n\n$(cat "$CONTEXT_FILE")\n\n---\n\nNew message: $prompt"
else
    echo "No previous context, using direct prompt..."
    FULL_PROMPT="$prompt"
fi

echo "Executing OpenCode..."
echo "Model: ${model:-default}"
echo "Prompt length: $(echo -e "$FULL_PROMPT" | wc -c) characters"
echo

# Execute OpenCode and capture output
OPENCODE_OUTPUT_FILE="$SESSION_DIR/latest_response.md"

if [ -n "$model" ]; then
    echo -e "$FULL_PROMPT" | opencode --model "$model" > "$OPENCODE_OUTPUT_FILE" 2>&1
else
    echo -e "$FULL_PROMPT" | opencode > "$OPENCODE_OUTPUT_FILE" 2>&1
fi

OPENCODE_EXIT_CODE=$?

if [ $OPENCODE_EXIT_CODE -eq 0 ]; then
    echo "✅ OpenCode execution successful"
else
    echo "❌ OpenCode execution failed (exit code: $OPENCODE_EXIT_CODE)"
    echo "Error output:"
    cat "$OPENCODE_OUTPUT_FILE"
    exit 1
fi

echo
echo "Response preview:"
echo "--- Start Response ---"
head -20 "$OPENCODE_OUTPUT_FILE"
echo "--- End Preview ---"
echo

In [ ]:
#!/bin/bash

# Update conversation history with new exchange
echo "=== Updating Conversation History ==="

MESSAGE_TIMESTAMP=$(date '+%Y-%m-%d %H:%M:%S')

# Add new message to conversation history
cat >> "$HISTORY_FILE" << EOF
## Message $NEW_MESSAGE_NUM - $MESSAGE_TIMESTAMP

**User:** $prompt

**OpenCode Response:**

$(cat "$OPENCODE_OUTPUT_FILE")

---

EOF

echo "✅ Conversation history updated with message $NEW_MESSAGE_NUM"

# Update session metadata
if command -v jq >/dev/null 2>&1; then
    # Update message count using jq
    jq --arg timestamp "$MESSAGE_TIMESTAMP" --arg count "$NEW_MESSAGE_NUM" \
       '.last_updated = $timestamp | .message_count = ($count | tonumber)' \
       "$METADATA_FILE" > "${METADATA_FILE}.tmp" && mv "${METADATA_FILE}.tmp" "$METADATA_FILE"
else
    # Fallback: update using sed
    sed -i "s/\"message_count\": [0-9]*/\"message_count\": $NEW_MESSAGE_NUM/" "$METADATA_FILE"
    sed -i "s/\"last_updated\": \"[^\"]*\"/\"last_updated\": \"$MESSAGE_TIMESTAMP\"/" "$METADATA_FILE"
fi

echo "✅ Session metadata updated"
echo

# Handle history trimming if needed
TOTAL_MESSAGES=$(grep -c "^## Message [0-9]" "$HISTORY_FILE")
if [ $TOTAL_MESSAGES -gt $max_history ]; then
    echo "History exceeds maximum ($TOTAL_MESSAGES > $max_history), trimming..."
    
    # Create a temporary file with header + last N messages
    # This is a simplified approach - production would need more robust handling
    head -n 10 "$HISTORY_FILE" > "${HISTORY_FILE}.tmp"
    
    # Add the last max_history messages
    grep -n "^## Message [0-9]" "$HISTORY_FILE" | tail -n $max_history | while IFS=: read -r line_num _; do
        tail -n +$line_num "$HISTORY_FILE" | head -n 50 >> "${HISTORY_FILE}.tmp"
    done
    
    mv "${HISTORY_FILE}.tmp" "$HISTORY_FILE"
    echo "✅ History trimmed to last $max_history messages"
else
    echo "History size within limits ($TOTAL_MESSAGES messages)"
fi

echo

In [ ]:
#!/bin/bash

# Session completion summary
echo "=== Session Summary ==="

FINAL_MESSAGE_COUNT=$(grep -c "^## Message [0-9]" "$HISTORY_FILE" 2>/dev/null || echo "0")
HISTORY_SIZE=$(wc -c < "$HISTORY_FILE" 2>/dev/null || echo "0")
SESSION_SIZE=$(du -sh "$SESSION_DIR" 2>/dev/null | cut -f1 || echo "unknown")

echo "Session ID: $session_id"
echo "Action completed: $action"
echo "Total messages: $FINAL_MESSAGE_COUNT"
echo "History file size: $HISTORY_SIZE bytes"
echo "Session directory size: $SESSION_SIZE"
echo "Session directory: $SESSION_DIR"
echo

echo "Session files:"
ls -la "$SESSION_DIR"
echo

echo "Current session metadata:"
cat "$METADATA_FILE"
echo

echo "✅ Multi-turn session management completed successfully!"
echo "💬 Session: $session_id"
echo "📝 Messages: $FINAL_MESSAGE_COUNT" 
echo "📁 Location: $SESSION_DIR"

# Show usage examples for continuing the session
echo
echo "🔄 To continue this session:"
echo "   make session SESSION_ID=$session_id PROMPT=\"Your next message\""
echo
echo "📊 To view session history:"
echo "   cat $HISTORY_FILE"
echo
echo "🏁 To end this session:"
echo "   make session ACTION=end SESSION_ID=$session_id"